In [2]:
install.packages("nycflights13")

library(tidyverse)
library(nycflights13)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [5]:
#assign individual databases from nycflights13 to variables
flights <- nycflights13::flights
airlines <- nycflights13::airlines
airports <- nycflights13::airports
planes <- nycflights13::planes
weather <- nycflights13::weather

In [6]:
glimpse(flights)
cat("\n")
glimpse(airlines)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(planes)
cat("\n")
glimpse(weather)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

Q1 least popular destination during the first quarter of the year

In [7]:
flights %>%
	filter(!is.na(dep_time), year == 2013, month %in% c(1,2,3)) %>%
	select(year, month, dest) %>%
  group_by(year, month, dest) %>%
  summarise(n_flights = n()) %>%
  arrange(n_flights) %>%
  head(5) %>%
  inner_join(airports %>%
                select(faa, name),
              by = c("dest" = "faa"))


`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


year,month,dest,n_flights,name
<int>,<int>,<chr>,<int>,<chr>
2013,1,EYW,1,Key West Intl
2013,1,AVL,2,Asheville Regional Airport
2013,1,JAC,2,Jackson Hole Airport
2013,2,JAC,2,Jackson Hole Airport
2013,3,BGR,2,Bangor Intl


Q2 airlines with most departure delay (on Christmas 2013)

In [8]:
flights %>%
  filter(year == 2013, month == 12, day == 25) %>%
  select(carrier, dep_delay) %>%
  arrange(desc(dep_delay)) %>%
  head(5) %>%
  inner_join(airlines, by = "carrier")

carrier,dep_delay,name
<chr>,<dbl>,<chr>
EV,321,ExpressJet Airlines Inc.
EV,251,ExpressJet Airlines Inc.
DL,234,Delta Air Lines Inc.
UA,198,United Air Lines Inc.
DL,193,Delta Air Lines Inc.


Q3 airline with least cancelled flights (compared to total flights)

In [9]:
na_deptime <- flights %>%
  filter(is.na(dep_time)) %>%
  group_by(carrier) %>%
  summarise(cancelled_flights = n())

flights %>%
  inner_join(airlines, by = "carrier") %>%
  group_by(carrier, name) %>%
  select(carrier, name) %>%
  summarise(scheduled_flights = n()) %>%
  inner_join(na_deptime, by = "carrier") %>%
  mutate(cancelled_flights_proportion =
    round((cancelled_flights/scheduled_flights), 4)) %>%
  arrange(cancelled_flights_proportion) %>%
  head(5)

`summarise()` has grouped output by 'carrier'. You can override using the
`.groups` argument.


carrier,name,scheduled_flights,cancelled_flights,cancelled_flights_proportion
<chr>,<chr>,<int>,<int>,<dbl>
AS,Alaska Airlines Inc.,714,2,0.0028
F9,Frontier Airlines Inc.,685,3,0.0044
VX,Virgin America,5162,31,0.0060
DL,Delta Air Lines Inc.,48110,349,0.0073
B6,JetBlue Airways,54635,466,0.0085


Q4 most used airplane (based from tail number)

In [10]:
flights %>%
  filter(!is.na(dep_time)) %>%
  group_by(tailnum) %>%
  select(carrier, tailnum) %>%
  summarise(departed_flights = n()) %>%
  arrange(desc(departed_flights)) %>%
  head(5)

tailnum,departed_flights
<chr>,<int>
N725MQ,546
N722MQ,487
N723MQ,480
N711MQ,467
N713MQ,455


Q5 most popular airplane manufacturer among airlines

In [11]:
planes %>%
  select(tailnum, manufacturer) %>%
  group_by(manufacturer) %>%
  summarise(no_of_airplanes = n()) %>%
  arrange(desc(no_of_airplanes)) %>%
  head(5)

manufacturer,no_of_airplanes
<chr>,<int>
BOEING,1630
AIRBUS INDUSTRIE,400
BOMBARDIER INC,368
AIRBUS,336
EMBRAER,299
